In [ ]:
# The purpose of this notebook is to find out what's the smallest network 
# that can be used to get max prediction score

In [ ]:
%run "standardScalerSetup.ipynb"
#loads all the preprocessing libraries and prepares the dataframe

In [ ]:
from sklearn import metrics

In [ ]:
# Neural Nets imports
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten 
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, BatchNormalization
from tensorflow.keras.optimizers import Adam, Adadelta, Adagrad, Adamax, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l1, l2, l1_l2

In [ ]:
verboseLevel=0
validationSplit=0.2
batchSize=30
epochs=1000

In [ ]:
# callback preparation
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.5,
                              patience=2,
                              verbose=verboseLevel,
                              mode='min',
                              min_lr=0.001)

In [ ]:
from tensorflow.keras.utils import multi_gpu_model

In [ ]:
inputSize = 9
colList = ['HiddenLayers', 'R2Score', 'MAE', 'MSE', 'H5FileName', 'TrainHistory', 'TrainPredictions']

In [ ]:
# Check Large Dataset first

In [ ]:
def createModel(dataFrame, layerSize, loops, y_train, X_train, y_test, X_test, targetScaler, labelSet):
    print(f'Creating model using layer size = {layerSize} on set = {labelSet}.\n')
    for i in range(loops):
        print(f'Training on {i} hidden layers\n')
        model = Sequential()
        model.add(Dense(layerSize, kernel_initializer='normal',
                        #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                        kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                        #kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),
                        input_dim=inputSize, activation=custom_activation))
        for j in range(i):
            model.add(Dense(layerSize, 
                            #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                            kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                            #kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),
                            kernel_initializer='normal', activation=custom_activation))
        model.add(BatchNormalization())
        model.add(Dense(1, kernel_initializer='normal', 
                        #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                        kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                        #kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),
                        activation='linear'))

        # only set this if GPU capabilities available
        # model = multi_gpu_model(model, gpus=2)

        optmzr=Adam(lr=0.001)    
        model.compile(optimizer=optmzr, loss='mae', metrics=['mae'])

        model_h5_name = 'mlp_' + str(layerSize)+ '_' + str(i) + '_model_std_' + labelSet + '_L2.h5'
        checkpoint_nn_std = ModelCheckpoint(model_h5_name,
                                 monitor='val_loss',
                                 verbose=verboseLevel,
                                 save_best_only=True,
                                 mode='min')
        callbacks_list_nn_std = [checkpoint_nn_std, reduce_lr]

        history_MLP_std = model.fit(X_train.to_numpy(), y_train,
                                    batch_size=batchSize, 
                                    validation_split=validationSplit, 
                                    epochs=epochs, verbose=verboseLevel,
                                    callbacks=callbacks_list_nn_std)

        #reload the best model!
        model_new = load_model(model_h5_name)
        #Predict
        y_pred_scaled = model_new.predict(X_test.to_numpy())
        #Evaluate metrics
        y_pred = targetScaler.inverse_transform(y_pred_scaled)
        r2_score = metrics.r2_score(y_test, y_pred)
        mae = metrics.mean_absolute_error(y_test, y_pred)
        mse = metrics.mean_squared_error(y_test, y_pred)
        #store values
        row = [i, r2_score, mae, mse, model_h5_name, history_MLP_std, y_pred]
        df = pd.DataFrame(np.array(row).reshape(1, len(colList)), columns=colList)
        dataFrame = dataFrame.append(df, ignore_index=True)

        tf.keras.backend.clear_session()
        del(model)
        del(model_new)
        
    return dataFrame


In [ ]:
%%time
dataFrame = pd.DataFrame(columns=colList)
layerSize = 64
loops = 15
dataFrame = createModel(dataFrame, layerSize, loops, 
                        y_train_scaled_std_all, X_train_all_std,
                        y_test_all_std, X_test_all_std, 
                        targetStdScalerAll, 'all')


In [ ]:
dataFrame

In [ ]:
#Plot train vs validation
plt.figure(figsize=(20,10))
#plt.plot(dataFrame['R2Score'])
plt.plot(dataFrame['MAE'])
#plt.plot(dataFrame['MSE'])
plt.title('Training Scores MLP')
plt.ylabel('Score')
plt.xlabel('Iteration')
plt.legend(['MAE'], loc='upper right')
plt.show()

In [ ]:
# Determine the IDX value where the MAE is smallest
minMaeIDX = dataFrame.loc[dataFrame['MAE']==dataFrame['MAE'].min()].index[0]

In [ ]:
dataFrame.iloc[minMaeIDX]

In [ ]:
minMaeAWS = dataFrame.iloc[minMaeIDX]['MAE']
minR2AWS = dataFrame.iloc[minMaeIDX]['R2Score']
modelNameAWS = "MLP_64_Std"
posAWS = minMaeIDX

In [ ]:
history_MLP = dataFrame['TrainHistory'][minMaeIDX]
#Plot train vs validation
plt.figure(figsize=(20,10))
plt.plot(history_MLP.history['loss'])
plt.plot(history_MLP.history['val_loss'])
plt.title('Validation vs Train loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
y_pred_MLP_std = dataFrame['TrainPredictions'][minMaeIDX]
# Plot prediction vs original
plt.figure(figsize=(20,10))
plt.scatter(range(y_test_all_std.shape[0]),y_test_all_std,label="Original Data", alpha=0.6, c='red')
plt.scatter(range(y_pred_MLP_std.shape[0]),y_pred_MLP_std,label="Predicted Data", 
            alpha=0.6, c='black')
plt.ylabel('Total Messages')
plt.xlabel('Records')
plt.title('MLP Std Model for X_test dataset prediction vs original')
plt.legend()
plt.show()

In [ ]:
%%time
dataFrame128 = pd.DataFrame(columns=colList)
layerSize = 128
#loops = 15
dataFrame128 = createModel(dataFrame128, layerSize, loops, 
                        y_train_scaled_std_all, X_train_all_std,
                        y_test_all_std, X_test_all_std, 
                        targetStdScalerAll, 'all')


In [ ]:
dataFrame128

In [ ]:
# Determine the IDX value where the MAE is smallest
minMaeIDX = dataFrame128.loc[dataFrame128['MAE']==dataFrame128['MAE'].min()].index[0]

In [ ]:
dataFrame128.iloc[minMaeIDX]

In [ ]:
if dataFrame128.iloc[minMaeIDX]['MAE'] < minMaeAWS :
    minMaeAWS = dataFrame128.iloc[minMaeIDX]['MAE']
    minR2AWS = dataFrame128.iloc[minMaeIDX]['R2Score']
    modelNameAWS = "MLP_128_Std"
    posAWS = minMaeIDX

In [ ]:
#Plot train vs validation
plt.figure(figsize=(20,10))
plt.rcParams.update({'font.size': 20})
#plt.plot(100*dataFrame128['R2Score'])
plt.plot(dataFrame128['MAE'])
#plt.plot(dataFrame128['MSE'])
plt.title('Training Scores MLP')
plt.ylabel('Score')
plt.xlabel('Iteration')
plt.legend(['MAE'], loc='upper right')
plt.show()

In [ ]:
history_MLP = dataFrame128['TrainHistory'][minMaeIDX]
#Plot train vs validation
plt.figure(figsize=(20,10))
plt.rcParams.update({'font.size': 20})
plt.plot(history_MLP.history['loss'])
plt.plot(history_MLP.history['val_loss'])
plt.title('Validation vs Train loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
y_pred_MLP_std = dataFrame128['TrainPredictions'][minMaeIDX]
# Plot prediction vs original
plt.figure(figsize=(20,10))
plt.rcParams.update({'font.size': 20})
plt.scatter(range(y_test_all_std.shape[0]),y_test_all_std,label="Original Data", alpha=0.6, c='red')
plt.scatter(range(y_pred_MLP_std.shape[0]),y_pred_MLP_std,label="Predicted Data", 
            alpha=0.6, c='black')
plt.ylabel('Total Messages')
plt.xlabel('Records')
plt.title('MLP Std Model for X_test dataset prediction vs original')
plt.rc('legend', fontsize=18)
plt.legend(loc='best', bbox_to_anchor=(0.75, 0, 0.25, 0.25))
plt.show()

In [ ]:
%%time
dataFrame32 = pd.DataFrame(columns=colList)
layerSize = 32
#loops = 15
dataFrame32 = createModel(dataFrame32, layerSize, loops, 
                        y_train_scaled_std_all, X_train_all_std,
                        y_test_all_std, X_test_all_std, 
                        targetStdScalerAll, 'all')


In [ ]:
# Determine the IDX value where the MAE is smallest
minMaeIDX = dataFrame32.loc[dataFrame32['MAE']==dataFrame32['MAE'].min()].index[0]

In [ ]:
dataFrame32.iloc[minMaeIDX]

In [ ]:
if dataFrame32.iloc[minMaeIDX]['MAE'] < minMaeAWS :
    minMaeAWS = dataFrame32.iloc[minMaeIDX]['MAE']
    minR2AWS = dataFrame32.iloc[minMaeIDX]['R2Score']
    modelNameAWS = "MLP_32_Std"
    posAWS = minMaeIDX

In [ ]:
#Plot train vs validation
plt.figure(figsize=(20,10))
plt.rcParams.update({'font.size': 20})
#plt.plot(100*dataFrame32['R2Score'])
plt.plot(dataFrame32['MAE'])
#plt.plot(dataFrame32['MSE'])
plt.title('Training Scores MLP')
plt.ylabel('Score')
plt.xlabel('Iteration')
plt.legend(['MAE'], loc='upper right')
plt.show()

In [ ]:
history_MLP = dataFrame32['TrainHistory'][minMaeIDX]
#Plot train vs validation
plt.figure(figsize=(20,10))
plt.rcParams.update({'font.size': 20})
plt.plot(history_MLP.history['loss'])
plt.plot(history_MLP.history['val_loss'])
plt.title('Validation vs Train loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
y_pred_MLP_std = dataFrame32['TrainPredictions'][minMaeIDX]
# Plot prediction vs original
plt.figure(figsize=(20,10))
plt.rcParams.update({'font.size': 20})
plt.scatter(range(y_test_all_std.shape[0]),y_test_all_std,label="Original Data", alpha=0.6, c='red')
plt.scatter(range(y_pred_MLP_std.shape[0]),y_pred_MLP_std,label="Predicted Data", 
            alpha=0.6, c='black')
plt.ylabel('Total Messages')
plt.xlabel('Records')
plt.title('MLP Std Model for X_test dataset prediction vs original')
plt.legend()
plt.show()

In [ ]:
#### Base Std

In [ ]:
%%time
dataFrame32_1 = pd.DataFrame(columns=colList)
layerSize = 32
#loops = 15
dataFrame32_1 = createModel(dataFrame32_1, layerSize, loops, 
                        y_train_scaled_std_base, X_train_base_std,
                        y_test1_summaryL1Z2_std, X_test1_summaryL1Z2_std, 
                        targetStdScalerBase, 'base')

In [ ]:
dataFrame32_1

In [ ]:
# Determine the IDX value where the MAE is smallest
minMaeIDX = dataFrame32_1.loc[dataFrame32_1['MAE']==dataFrame32_1['MAE'].min()].index[0]

In [ ]:
dataFrame32_1.iloc[minMaeIDX]

In [ ]:
#if dataFrame32_1.iloc[minMaeIDX]['MAE'] < minMaeAWS :
minMaeAWS_base = dataFrame32_1.iloc[minMaeIDX]['MAE']
minR2AWS_base = dataFrame32_1.iloc[minMaeIDX]['R2Score']
modelNameAWS_base = "MLP_32_Base_Std"
posAWS_base = minMaeIDX

In [ ]:
#Plot train vs validation
plt.figure(figsize=(20,10))
#plt.plot(100*dataFrame32['R2Score'])
plt.plot(dataFrame32_1['MAE'])
#plt.plot(dataFrame32['MSE'])
plt.title('Training Scores MLP')
plt.ylabel('Score')
plt.xlabel('Iteration')
plt.legend(['MAE'], loc='upper right')
plt.show()

In [ ]:
history_MLP = dataFrame32_1['TrainHistory'][minMaeIDX]
#Plot train vs validation
plt.figure(figsize=(20,10))
plt.plot(history_MLP.history['loss'])
plt.plot(history_MLP.history['val_loss'])
plt.title('Validation vs Train loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
y_pred_MLP_std = dataFrame32_1['TrainPredictions'][minMaeIDX]
# Plot prediction vs original
plt.figure(figsize=(20,10))
plt.scatter(range(y_test1_summaryL1Z2_std.shape[0]),y_test1_summaryL1Z2_std,label="Original Data", alpha=0.6, c='red')
plt.scatter(range(y_pred_MLP_std.shape[0]),y_pred_MLP_std,label="Predicted Data", 
            alpha=0.6, c='black')
plt.ylabel('Total Messages')
plt.xlabel('Records')
plt.title('MLP Std Model for X_test dataset prediction vs original')
plt.legend()
plt.show()

In [ ]:
%%time
dataFrame64_1 = pd.DataFrame(columns=colList)
layerSize = 64
#loops = 15
dataFrame64_1 = createModel(dataFrame64_1, layerSize, loops, 
                        y_train_scaled_std_base, X_train_base_std,
                        y_test1_summaryL1Z2_std, X_test1_summaryL1Z2_std, 
                        targetStdScalerBase, 'base')


In [ ]:
# Determine the IDX value where the MAE is smallest
minMaeIDX = dataFrame64_1.loc[dataFrame64_1['MAE']==dataFrame64_1['MAE'].min()].index[0]

In [ ]:
dataFrame64_1.iloc[minMaeIDX]

In [ ]:
if dataFrame64_1.iloc[minMaeIDX]['MAE'] < minMaeAWS_base :
    minMaeAWS_base = dataFrame64_1.iloc[minMaeIDX]['MAE']
    minR2AWS_base = dataFrame64_1.iloc[minMaeIDX]['R2Score']
    modelNameAWS_base = "MLP_64_Base_Std"
    posAWS_base = minMaeIDX

In [ ]:
#Plot train vs validation
plt.figure(figsize=(20,10))
#plt.plot(100*dataFrame32['R2Score'])
plt.plot(dataFrame64_1['MAE'])
#plt.plot(dataFrame32['MSE'])
plt.title('Training Scores MLP')
plt.ylabel('Score')
plt.xlabel('Iteration')
plt.legend(['MAE'], loc='upper right')
plt.show()

In [ ]:
history_MLP = dataFrame64_1['TrainHistory'][minMaeIDX]
#Plot train vs validation
plt.figure(figsize=(20,10))
plt.plot(history_MLP.history['loss'])
plt.plot(history_MLP.history['val_loss'])
plt.title('Validation vs Train loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
y_pred_MLP_Std = dataFrame64_1['TrainPredictions'][minMaeIDX]
# Plot prediction vs original
plt.figure(figsize=(20,10))
plt.scatter(range(y_test1_summaryL1Z2_std.shape[0]),y_test1_summaryL1Z2_std,label="Original Data", alpha=0.6, c='black')
plt.scatter(range(y_pred_MLP_std.shape[0]),y_pred_MLP_std,label="Predicted Data", 
            alpha=0.6, c='red')
plt.ylabel('Total Messages')
plt.xlabel('Records')
plt.title('MLP Std Model for X_test dataset prediction vs original')
plt.legend()
plt.show()

In [ ]:
%%time
dataFrame128_1 = pd.DataFrame(columns=colList)
layerSize = 128
#loops = 15
dataFrame128_1 = createModel(dataFrame128_1, layerSize, loops, 
                        y_train_scaled_std_base, X_train_base_std,
                        y_test1_summaryL1Z2_std, X_test1_summaryL1Z2_std, 
                        targetStdScalerBase, 'base')


In [ ]:
# Determine the IDX value where the MAE is smallest
minMaeIDX = dataFrame128_1.loc[dataFrame128_1['MAE']==dataFrame128_1['MAE'].min()].index[0]

In [ ]:
dataFrame128_1.iloc[minMaeIDX]

In [ ]:
if dataFrame128_1.iloc[minMaeIDX]['MAE'] < minMaeAWS_base :
    minMaeAWS_base = dataFrame128_1.iloc[minMaeIDX]['MAE']
    minR2AWS_base = dataFrame128_1.iloc[minMaeIDX]['R2Score']
    modelNameAWS_base = "MLP_64_Base_Std"
    posAWS_base = minMaeIDX

In [ ]:
#Plot train vs validation
plt.figure(figsize=(20,10))
#plt.plot(100*dataFrame32['R2Score'])
plt.plot(dataFrame128_1['MAE'])
#plt.plot(dataFrame32['MSE'])
plt.title('Training Scores MLP')
plt.ylabel('Score')
plt.xlabel('Iteration')
plt.legend(['MAE'], loc='upper right')
plt.show()

In [ ]:
history_MLP = dataFrame128_1['TrainHistory'][minMaeIDX]
#Plot train vs validation
plt.figure(figsize=(20,10))
plt.plot(history_MLP.history['loss'])
plt.plot(history_MLP.history['val_loss'])
plt.title('Validation vs Train loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
y_pred_MLP_Std = dataFrame128_1['TrainPredictions'][minMaeIDX]
# Plot prediction vs original
plt.figure(figsize=(20,10))
plt.scatter(range(y_test1_summaryL1Z2_std.shape[0]),y_test1_summaryL1Z2_std,label="Original Data", alpha=0.6, c='black')
plt.scatter(range(y_pred_MLP_std.shape[0]),y_pred_MLP_std,label="Predicted Data", 
            alpha=0.6, c='red')
plt.ylabel('Total Messages')
plt.xlabel('Records')
plt.title('MLP Std Model for X_test dataset prediction vs original')
plt.legend()
plt.show()

In [ ]:
minMaeAWS, minR2AWS, modelNameAWS, posAWS

In [ ]:
minMaeAWS_base, minR2AWS_base, modelNameAWS_base, posAWS_base